# Project 1, Part 3, Executive Questions: Product Specific Questions

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: Sophie Yeh

Year: 2022

Semester: Spring

Section: 8


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [35]:
!pip install plotly
import math
import numpy as np
import pandas as pd
import psycopg2
import plotly.express as px

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [36]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
#connect to postgres database
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
# create cursor for connections
cursor = connection.cursor()

# Instructions for Specific Questions

For each specific quesion, you must write 1 and only 1 query that answers that specific question.  The results of your query must be displayed in a Pandas dataframe as demonstrated in the labs

All output should be sorted.

For store name, use the city name the store is located in.

When sorting by store name, sort in alphabetical order.

For meal name, use the product description.

When sorting by meal name, sort in alphabetical order.

For month, use the full month name: January, February, March, etc.

When sorting by month, sort in calendar order: January, February, March, etc.  You may find it useful to include a month number to help with the sort.  It's ok to leave it in the output.

For day of week, use the full day of week name: Sunday, Monday, Tuesday, etc.

When sorting by day of week, sort in day of week order: Sunday, Monday, Tuesday, etc. You may find it usefule to include a day of week number to help with the sort.  It's ok to leave it in the output.


# 1.3.1 How many meals were purchased for all of AGM?

In [37]:
def ct_meals_agm():
    # Meals purchased for all of AGM
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT SUM(quantity) as quantity
    FROM line_items

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
ct_meals_agm().style.set_caption('Meals purchased for all of AGM')

,quantity
0,8228284


# 1.3.2 How many meals were purchased for all of AGM by meal?

Sort by meal name in alphabetical order

In [38]:
def ct_meals_by_meal():
    # Meals purchased for all of AGM by meal
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT prod.description as meal_name, SUM(li.quantity) as quantity
    FROM line_items as li
        JOIN products as prod
        ON li.product_id = prod.product_id
    GROUP BY meal_name
    ORDER BY meal_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
ct_meals_by_meal().style.set_caption('Meals purchased for all of AGM by meal')

,meal_name,quantity
0,Brocolli Stir Fry,913984
1,Chicken Salad,228561
2,Curry Chicken,1368884
3,Eggplant Lasagna,1599058
4,Pistachio Salmon,1828778
5,Spinach Orzo,456769
6,Teriyaki Chicken,1145013
7,Tilapia Piccata,687237


# 1.3.3 How many meals were purchased by store and by meal?

Sort by store name in alphabetical order, then by meal name in alphabetical order

In [39]:
def ct_meals_by_store_meal():
    # Meals purchased by store and meal
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT s.city as store, prod.description as meal_name, SUM(li.quantity) as quantity
    FROM line_items as li
        JOIN products as prod
        ON li.product_id = prod.product_id
        JOIN stores as s
        ON li.store_id = s.store_id
    GROUP BY store, meal_name
    ORDER BY store, meal_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

ct_meals_by_store_meal().style.set_caption('Meals purchased by store and meal')

,store,meal_name,quantity
0,Berkeley,Brocolli Stir Fry,232038
1,Berkeley,Chicken Salad,57719
2,Berkeley,Curry Chicken,346508
3,Berkeley,Eggplant Lasagna,405637
4,Berkeley,Pistachio Salmon,464274
5,Berkeley,Spinach Orzo,115469
6,Berkeley,Teriyaki Chicken,290858
7,Berkeley,Tilapia Piccata,174252
8,Dallas,Brocolli Stir Fry,179885
9,Dallas,Chicken Salad,44756


# 1.3.4 How many meals were purchased by month?

Sort by month in calendar order

In [40]:
def ct_meals_by_month():
    # Meals purchased by month
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT EXTRACT(MONTH FROM sa.sale_date) as month_num,
           TO_CHAR(sa.sale_date, 'Month') as month_name, 
           SUM(li.quantity) as quantity
    FROM line_items as li
        JOIN sales as sa
        ON li.sale_id = sa.sale_id
        AND li.store_id = sa.store_id
    GROUP BY month_num, month_name
    ORDER BY month_num, month_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
ct_meals_by_month().style.set_caption('Meals purchased by month')

,month_num,month_name,quantity
0,1,January,650319
1,2,February,631190
2,3,March,731635
3,4,April,687607
4,5,May,664820
5,6,June,677009
6,7,July,666087
7,8,August,752484
8,9,September,631580
9,10,October,741259


# 1.3.5 How many meals were purchased by month and meal?

Sort by month in calendar order, then by meal name in alphabetical order

In [41]:
def meals_by_month_meal():
    # meals purchased by month and meal
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT EXTRACT(MONTH FROM sa.sale_date) as month_num, 
           TO_CHAR(sa.sale_date, 'Month') as month_name, 
           prod.description as meal_name, 
           SUM(li.quantity) as quantity
    FROM line_items as li
        JOIN sales as sa
        ON li.sale_id = sa.sale_id
        AND li.store_id = sa.store_id
        JOIN products as prod
        ON li.product_id = prod.product_id
    GROUP BY month_num, month_name, meal_name
    ORDER BY month_num, month_name, meal_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

meals_by_month_meal().style.set_caption('Meals purchased by month and meal')

,month_num,month_name,meal_name,quantity
0,1,January,Brocolli Stir Fry,72161
1,1,January,Chicken Salad,18003
2,1,January,Curry Chicken,108134
3,1,January,Eggplant Lasagna,126605
4,1,January,Pistachio Salmon,144053
5,1,January,Spinach Orzo,36262
6,1,January,Teriyaki Chicken,90618
7,1,January,Tilapia Piccata,54483
8,2,February,Brocolli Stir Fry,70164
9,2,February,Chicken Salad,17597


# 1.3.6 How many meals were purchased by day of week and meal?

Sort by day of week in day of week order, then by meal name in alphabetical order

In [42]:
def meals_by_dow_meal():
    # Meals purchased by day of week and meal
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT EXTRACT(dow FROM sa.sale_date) as dow, 
           TO_CHAR(sa.sale_date, 'Day') as day_of_week, 
           prod.description as meal_name, 
           SUM(li.quantity) as quantity
    FROM line_items as li
        JOIN sales as sa
        ON li.sale_id = sa.sale_id
        AND li.store_id = sa.store_id
        JOIN products as prod
        ON li.product_id = prod.product_id
    GROUP BY dow, day_of_week, meal_name
    ORDER BY dow, day_of_week, meal_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

meals_by_dow_meal().style.set_caption('Meals purchased by day of week and meal')

,dow,day_of_week,meal_name,quantity
0,0,Sunday,Brocolli Stir Fry,172250
1,0,Sunday,Chicken Salad,43110
2,0,Sunday,Curry Chicken,257216
3,0,Sunday,Eggplant Lasagna,301726
4,0,Sunday,Pistachio Salmon,344315
5,0,Sunday,Spinach Orzo,85968
6,0,Sunday,Teriyaki Chicken,215741
7,0,Sunday,Tilapia Piccata,128763
8,1,Monday,Brocolli Stir Fry,121717
9,1,Monday,Chicken Salad,30790


# 1.3.7 Average number of meals per sale for all of AGM


In [43]:
def avg_meals_per_sale_agm():
    # Average number of meals per sale for all of AGM
    rollback_before_flag = True
    rollback_after_flag = True

    query = """
    SELECT SUM(quantity)/COUNT(DISTINCT sale_id) as avg_quantity
    FROM line_items
    
    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
avg_meals_per_sale_agm().style.set_caption('Average number of meals per sale for all of AGM')

,avg_quantity
0,21.077897


# 1.3.8 Average number of meals per sale by store

Sort by store name in alphabetical order


In [44]:
def avg_meals_per_sale_by_store():
    # Average number of meals per sale by store
    rollback_before_flag = True
    rollback_after_flag = True

    query = """

    SELECT s.city as store, SUM(l.quantity)/COUNT(DISTINCT l.sale_id) as avg_quantity
    FROM line_items as l
        JOIN stores as s 
        ON l.store_id = s.store_id
    GROUP BY s.city
    ORDER BY s.city
    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
avg_meals_per_sale_by_store().style.set_caption('Average number of meals per sale by store')

,store,avg_quantity
0,Berkeley,5.345514
1,Dallas,5.353353
2,Miami,5.359892
3,Nashville,5.332978
4,Seattle,5.361470


# 1.3.9 The Executives have also asked you to provide your best example of a data visualization for one of the above queries

You may choose any one of the above queries. 

It's best to select the one that you think will provide the most interesting data visualization.  

You are allowed to add a pivot as demonstrated in the labs if it is needed for your data visualization. 

In [45]:
def fig_meals_by_meal():
    # count of meals by meal ranking
    fig = px.bar(ct_meals_by_meal().sort_values('quantity'), x='quantity', y='meal_name',
                title='Quantity of AGM Meals',
                template='plotly_white',
                orientation='h',
                labels={'meal_name':'meal'},
                hover_data=['meal_name','quantity'],
                text_auto='.2s')
    fig.show()
    return
fig_meals_by_meal()